In [1]:

library('hdp')
library('clusterCrit')
library('grid')
library('gridExtra')
library('ggplot2')
library('ggrepel')

source('../../../src/tools.R')     # custom tools function
source('../../../src/hdp_tools.R') # hdp related functions
source('../../../src/hdp_tools_yanis.R')
theme_set(theme_minimal())

# set jupyer notebook parameters
options(repr.plot.res        = 100, # set a medium-definition resolution for the jupyter notebooks plots (DPI)
        repr.matrix.max.rows = 200, # set the maximum number of rows displayed
        repr.matrix.max.cols = 200) # set the maximum number of columns displayed

Run citation('hdp') for citation instructions,
    and file.show(system.file('LICENSE', package='hdp')) for license details.


### Some analysis of the dataframe to get familiar

In [84]:

df_final <- read.table("../../../data/updated_dataset/modif_final.csv",sep = ',' , header = T)
rownames(df_final)<- df_final$data_pd
df_final <- df_final[,-1:-3]
df_to_recluster <-read.table("df_to_recluster.csv",sep = ',' , header = T)
df_to_recluster <- df_to_recluster[colSums(df_to_recluster) > 0]

In [48]:
head(df_to_recluster)

,ASXL1,ASXL2,ASXL3,ATRX,BAGE3,BCOR,BRAF,CBFB,CBL,CDKN2A,CEBPA_bi,CEBPA_mono,CNTN5,CREBBP,CSF1R,CSF3R,CTCF,CUL2,CUX1,DNMT3A,EED,ETV6,EZH2,FBXW7,ITD,FLT3_TKD,FLT3_other,GATA1,GATA2,GNAS,GNB1,IDH1,IDH2_p.R140,IDH2_p.R172,JAK2,JAK3,KANSL1,KDM6A,KIT,KMT2C,KMT2D,KMT2E,KRAS,LUC7L2,MED12,MLL,MPL,MYC,NF1,NFE2,NOTCH1,NPM1,NRAS_other,NRAS_p.G12_13,NRAS_p.Q61_62,PDS5B,PHF6,PPFIA2,PRPF8,PTEN,PTPN11,PTPRF,PTPRT,RAD21,RIT1,RUNX1,S100B,SETBP1,SF1,SF3B1,SMC1A,SMC3,SMG1,SPP1,SRSF2,STAG2,STAT5B,SUZ12,TET2,TP53,U2AF1,WT1,ZRSR2,add_8,add_11,add_13,add_21,add_22,del_20,del_3,del_5,del_7,del_9,del_12,del_13,del_16,del_17,del_18,minusy,t_v_11,t_10_21,t_12_13,t_12_17,t_12_22,t_13_19,t_15_16,t_15_17,t_16_17,t_16_21,t_17_19,t_17_21,t_1_12,t_1_14,t_1_16,t_1_17,t_1_19,t_1_3,t_1_4,t_1_5,t_1_6,t_2_17,t_2_3,t_2_5,t_2_7,t_2_9,t_3_16,t_3_21,t_3_5,t_3_7,t_3_9,t_4_12,t_4_21,t_4_9,t_5_12,t_5_17,t_5_9,t_6_9,t_7_16,t_7_17,t_7_8,t_8_10,t_8_13,t_8_16,t_8_17,t_8_21,t_9_11,t_9_13,t_9_17,t_9_22,complex,others_transloc
PD20337c,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
PD20336a,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
PD20334a,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
PD20325a,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
PD20324a,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
PD20323a,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
tail(df_final)

,ASXL1,ASXL2,ASXL3,ATRX,BAGE3,BCOR,BRAF,CBFB,CBL,CDKN2A,CEBPA_bi,CEBPA_mono,CNTN5,CREBBP,CSF1R,CSF3R,CTCF,CUL2,CUX1,DNMT3A,EED,ETV6,EZH2,FBXW7,ITD,FLT3_TKD,FLT3_other,GATA1,GATA2,GNAS,GNB1,IDH1,IDH2_p.R140,IDH2_p.R172,JAK2,JAK3,KANSL1,KDM6A,KIT,KMT2C,KMT2D,KMT2E,KRAS,LUC7L2,MED12,MLL,MPL,MYC,NF1,NFE2,NOTCH1,NPM1,NRAS_other,NRAS_p.G12_13,NRAS_p.Q61_62,PDS5B,PHF6,PPFIA2,PRPF8,PTEN,PTPN11,PTPRF,PTPRT,RAD21,RIT1,RUNX1,S100B,SETBP1,SF1,SF3B1,SMC1A,SMC3,SMG1,SPP1,SRSF2,STAG2,STAT5B,SUZ12,TET2,TP53,U2AF1,WT1,ZRSR2,add_8,add_11,add_13,add_21,add_22,del_20,del_3,del_5,del_7,del_9,del_12,del_13,del_16,del_17,del_18,minusy,t_v_11,t_10_21,t_12_13,t_12_17,t_12_22,t_13_19,t_15_16,t_15_17,t_16_17,t_16_21,t_17_19,t_17_21,t_1_12,t_1_14,t_1_16,t_1_17,t_1_19,t_1_3,t_1_4,t_1_5,t_1_6,t_2_17,t_2_3,t_2_5,t_2_7,t_2_9,t_3_16,t_3_21,t_3_5,t_3_7,t_3_9,t_4_12,t_4_21,t_4_9,t_5_12,t_5_17,t_5_9,t_6_9,t_7_16,t_7_17,t_7_8,t_8_10,t_8_13,t_8_16,t_8_17,t_8_21,t_9_11,t_9_13,t_9_17,t_9_22,complex,others_transloc
PD25623a,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
PD25624c,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
PD25625a,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
PD25626c,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
PD25627a,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
PD25629c,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [63]:
dim(df_final)

[1] 658 151

In [85]:
dim(df_to_recluster)

[1] 658 115

### Clustering HDP

## LETS WORK ON THE GRIDSEARCH

### Base Distribution Functions

In [65]:
ncol(df_final)

[1] 151

In [86]:
###Binomial
num_cols = ncol(df_to_recluster)
bin <- function(x){
    set.seed(123)
  (rbinom(1, num_cols, mean(x))+1)/num_cols
}

###Normal

normal <- function(x){
    set.seed(123)
  abs(rnorm(1,mean(x),sd(x)))
}

###Poisson

poisson <- function(x){
    set.seed(123)
  (rpois(num_cols,1)+1)/num_cols
}

###Uniform equally over all columns

equally <- function(x){
    set.seed(123)
  1/num_cols
}

###Repet 1

repet <- function(x){
    set.seed(123)
  1
}

#sapply(df_gene,equally)
binomial <- unlist(sapply(df_to_recluster,bin))
gaussian <- unlist(sapply(df_to_recluster,normal))
pois <- as.numeric(unlist(sapply(df_to_recluster,poisson)))
unif <- unlist(sapply(df_to_recluster,equally))
repetition <- unlist(sapply(df_to_recluster,repet))
lists<- list(binomial,gaussian,unif,repetition)
#for (lis in lists) {
    #initialise_hdp_yanis(data=df_final,alphaa=1,alphab=1,hh=lis)
#}

In [95]:
# define gridsearch function
test_assignment <- function(x, dd_predicted, components_sd){
    if(x['second_predicted_component']%in%dd_predicted$predicted_component) {
        return( x['max_proba'] > x['second_max_proba']+components_sd[[x['second_predicted_component']+1]])
    }
    else {return(TRUE)} 
}
hdp_gridsearch_yanis <- function( data, posterior_samples, initial_clusters, burnin, chains, space, base_dist, alphaa, alphab){  #+3 components

    result_table <- data.frame('chains' = integer(),
                              'inicc' = integer(),
                              'n' = integer(),
                              'burnin' = integer(),
                              'space' = integer(),
                              'base_dist' = character(), 
                              'alphaa' = integer(),
                              'alphab' = integer(),
                              'n_components' = integer(),
                              'component_0' = integer(),
                              'assignment' = numeric(),
                              'Davies_Bouldin' =numeric(),
                              'Silhouette' = numeric(),stringsAsFactors=FALSE)
    
    
    for (a in chains) {
        for (b in initial_clusters) {
            for (c in posterior_samples) {
                for (d in burnin) {
                    for (e in space) {
                        for (f in base_dist){
                            for (g in alphaa){
                                for (h in alphab){
                                    str <-""
                                    if (f[1] == binomial[1]){
                                        str <-"binomial"
                                    } else if (f[1] == gaussian[1]){
                                        str <-"gaussian"
                                    } else if(f[1] == unif[1]){
                                        str <-"uniform"
                                    } else{
                                        str <-"repetition"
                                    }
                                    cat(sprintf('### %d chains | %d initial clusters | %d posterior samples | %d burn-in | %d space ###\n', a, b, c, d, e))
                                    flush.console()

                                    number_of_chains <- a
                                    chain_list <- vector('list', number_of_chains)
                                    hdp <- initialise_hdp_yanis(data=data,alphaa=g,alphab=h,hh=f)
                                    for (i in 1:number_of_chains) {
                                        seed <- i * 100

                                        activated_hdp <- dp_activate(hdp,
                                            dpindex = 1:numdp(hdp), # indices of the DP to activate: we choose all DP (1, ..., n + 1)
                                            initcc  = b,           # number of starting clusters (every data item is randomly assigned to a cluster to start with)
                                            seed    = seed)

                                        hdp_output <- hdp_posterior(activated_hdp,
                                            burnin = d, # number of burn-in iterations
                                            n      = c,      # number of posterior samples to collect
                                            space  = e,  # number of iterations between collected samples
                                            # cpiter = 1,    # The number of iterations of concentration parameter sampling to perform after each iteration.
                                            seed   = seed)

                                        chain_list[[i]] <- hdp_output

                                    }

                                    multi_output <- hdp_multi_chain(chain_list)
                                    multi_output <- hdp_extract_components(multi_output)


                                    dd_predicted <- data.frame(comp_dp_distn(multi_output)$mean[-1,])

                                    # change categories colnames
                                    colnames(dd_predicted) <- paste0('component_', 0:(ncol(dd_predicted)-1))
                                    components_colnames <- colnames(dd_predicted)

                                    # evaluate for each row the predicted component
                                    dd_predicted['predicted_component'] <- apply(dd_predicted, 1, function(x) { if (all(is.na(x)))
                                                                                                     return(NaN)
                                                                                                else
                                                                                                     return(which.max(x)-1)
                                                                                                })




                                    # evaluate for each row the maximum probability associated to the predicted component
                                    dd_predicted['max_proba'] <- apply(dd_predicted[,components_colnames], 1, function(x) { if (all(is.na(x)))
                                                                                                     return(NaN)
                                                                                                else
                                                                                                     return(max(x))})


                                    # get second predicted component
                                    n_components <- ncol(dd_predicted) - 2 - 1 # without columns 'component_0', 'predicted_component' and 'max_proba'

                                    dd_predicted['second_max_proba'] <- apply(dd_predicted[,0:n_components + 1], 1, function(x) { if (all(is.na(x)))
                                                                                                     return(NaN)
                                                                                                else
                                                                                                     return(sort(x,partial=n_components)[n_components])})

                                    dd_predicted['second_predicted_component'] <- apply(dd_predicted, 1, function(x) { if (all(is.na(x)))
                                                                                                     return(NaN)
                                                                                                else
                                                                                                     return(which(x==x['second_max_proba'])[1]-1)})



                                    # get standard deviations of components
                                    components_sd <- list()
                                    for( j in as.integer(levels(factor(dd_predicted$predicted_component)))){
                                        if(! is.na(j)){
                                            components_sd[[j+1]] <- sd(dd_predicted$max_proba[!is.na(dd_predicted$predicted_component) & dd_predicted$predicted_component == j])
                                        }
                                    }


                                    # determine which patients are well assigned
                                    dd_predicted['assignment'] <- apply(dd_predicted[, c('max_proba', 'second_max_proba','second_predicted_component')], 1, function(x) { if (all(is.na(x)))
                                                                                                     return(NaN)
                                                                                                else
                                                                                                     return(test_assignment(x, dd_predicted, components_sd))})

                                    # validation indexes
                                    traj <- data.frame(data[rowSums(data) !=0,])
                                    traj <- as.matrix(apply(traj, 2, function (x){as.numeric(x)}))
                                    part <- as.integer(dd_predicted$predicted_component[!is.na(dd_predicted$predicted_component)])
                                    intIdx <- intCriteria(traj, part, c("Davies_bouldin","Silhouette"))

                                    # result
                                    result_table[nrow(result_table)+1,] <- c(a, b, c, d, e, str, g, h, numcomp(multi_output), nrow(dd_predicted[!is.na(dd_predicted$predicted_component) &dd_predicted$predicted_component==0,]), nrow(dd_predicted[!is.na(dd_predicted$assignment) & dd_predicted$assignment==TRUE,]),intIdx$davies_bouldin,intIdx$silhouette)

                                    cat('Done!\n')
                                    flush.console()
                                }
                            }
                        }
                    }
                }
            }
        }
    }
    
    return(result_table)

}

In [93]:
# initialize the range of parameters used for the grid search
data <- df_to_recluster
posterior_samples <- c(350,450)
initial_clusters <- c(3,4,5,6,7)
burnin <- 7000
chains <- 3
space <- 20
base_dist <- gaussian
alphaa <-2
alphab <-0.5



In [94]:
ress <- hdp_gridsearch_yanis(data=df_to_recluster, posterior_samples, initial_clusters, burnin, chains, space,base_dist,alphaa,alphab)
ress

### 3 chains | 3 initial clusters | 350 posterior samples | 7000 burn-in | 20 space ###
Initialise HDP on a 658 x 115 dataframe
  → create HDP structure... done!
  → add DP node for each patient... done!
  → assign the data to the nodes...

ERROR: Error in hdp_setdata(hdp, dpindex = (1:nrow(data)) + 1, data = data): ncol(data) must equal numcateg(hdp)


In [15]:
write.table(ress,'hdp_gridsearch_rep_gauss_binomial_bis.tsv',sep='\t',quote=F)